In [14]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

driver = webdriver.Chrome()  # Optional argument, if not specified will search path.
base_page = 'http://www14.tceq.texas.gov/iwud/dist/index.cfm?fuseaction=ListDistricts&Command=list'
driver.get(base_page)
#CountyCode

audit_vars_css = 'td.iwud:nth-child(1)'
audit_vals_css = '.iwud+ td'
get_doc_id = 'table:nth-child(6) tr:nth-child(1) .iwuddata'
access_more_docs = '.iwudgreyrow~ tr a img'
wd_info_call = '.iwuddata:nth-child(2)'
#wd_info_header = 'tr:nth-child(5) .iwudheading , .iwudheading:nth-child(1)'
wd_info_header = '.iwudheading:nth-child(1)'
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
#

#empty_reference_df = pd.DataFrame()
#empty_district_info = pd.DataFrame()
empty_district_info = pd.read_csv('../../input/tceq_audits/district_info.csv',index_col=0)
empty_reference_df = pd.read_csv('../../input/tceq_audits/master_district_reference.csv',index_col=0)
        
should_restart = True
while should_restart:
    should_restart = False
    district_links = driver.find_elements_by_css_selector('table+ table .iwud a')
    district_ids = driver.find_elements_by_css_selector('.iwud+ .iwud:nth-child(3)')
    district_status = driver.find_elements_by_css_selector('.iwud~ .iwud+ .iwud')
    district_name_list = []
    district_id_list = []
    district_status_list = []
    district_link_list = []
    for district in district_links:
        district_name_list.append(district.text)
        district_link_list.append(district.get_attribute('href'))
    for ids in district_ids:
        district_id_list.append(ids.text)
    for status in district_status:
        district_status_list.append(status.text)  
    temp_district_set = pd.DataFrame({'SYSTEM_NAME': district_name_list,
     'URL': district_link_list,'Status':district_status_list,'SYSTEM_ID':district_id_list})
    empty_reference_df = empty_reference_df.append(temp_district_set)
    for district in range(len(district_links)):
        if empty_district_info.shape[0]>0:
            if any(district_ids[district].text == empty_district_info['SYSTEM_ID'].apply(str)):
                if (district+1) == len(district_links):
                    if len(driver.find_elements_by_css_selector("img[alt='Next page']"))!=0:
                        driver.find_element_by_name('next').click()
                        district_links = driver.find_elements_by_css_selector('table+ table .iwud a')
                        should_restart = True
                        continue
                continue
        print(district_links[district].text)
        district_links[district].click() 
        info_vars = []
        info = []
        for el in driver.find_elements_by_css_selector(wd_info_call):
            info.append(el.text)
        for el in driver.find_elements_by_css_selector(wd_info_header):
            info_vars.append(el.text)
        i = 0
        for v in range(len(info_vars)):
            if info_vars[v] == '':
                i = i + 1
                info_vars[v] = str(i)
            #print(info_vars);print(info)
        temp = pd.DataFrame(info).T
        temp.columns = [info_vars]
        temp['NAME'] = district_name_list[district]
        temp['SYSTEM_ID'] = district_id_list[district]
        while sum(temp.columns.duplicated())>0:
            temp = temp.drop(temp.columns[temp.columns.duplicated()][0],axis=1)
        empty_district_info = pd.concat([empty_district_info,temp], axis=0, join='outer', ignore_index=True)
        driver.back()
        district_links = driver.find_elements_by_css_selector('table+ table .iwud a')
        district_ids = driver.find_elements_by_css_selector('.iwud+ .iwud:nth-child(3)')
        if (district+1) == len(district_links):
            if len(driver.find_elements_by_css_selector("img[alt='Next page']"))!=0:
                driver.find_element_by_name('next').click()
                district_links = driver.find_elements_by_css_selector('table+ table .iwud a')
                district_ids = driver.find_elements_by_css_selector('.iwud+ .iwud:nth-child(3)')
                should_restart = True
        empty_district_info.to_csv('../../input/tceq_audits/district_info.csv')
        empty_reference_df.to_csv('../../input/tceq_audits/master_district_reference.csv')

LEANDER MUD 3
LEANDER TODD MUD 1
LEE COUNTY FWSD 1
LEE COUNTY WEST YEGUA WCID 2
LEE-FAYETTE COUNTIES CUMMINS CREEK WCID 1
LEE-FAYETTE-COLORADO COUNTIES CUMMINS CREEK WCID 1
LEGACY MUD 1
LEON RIVER FLOOD CONTROL DISTRICT
LERIN HILLS MUD
LIBERTY COUNTY CONSERVATION RECLAMATION DIST 3
LIBERTY COUNTY DRAINAGE DISTRICT 2
LIBERTY COUNTY DRAINAGE DISTRICT 3
LIBERTY COUNTY DRAINAGE DISTRICT 4
LIBERTY COUNTY DRAINAGE DISTRICT 5
LIBERTY COUNTY MUD 2
LIBERTY COUNTY MUD 3
LIBERTY COUNTY MUD 5
LIBERTY COUNTY MUD 6
LIBERTY COUNTY WCID 1
LIBERTY COUNTY WCID 2
LIBERTY COUNTY WCID 5
LIBERTY COUNTY WCID 6
LIBERTY COUNTY WCID 8
LIBERTY LAKES FWSD 1
LIBERTY PUD
LIBERTY-DANVILLE FRESH WATER SUPPLY DISTRICT 2
LIBERTY-DANVILLE FWSD 1
LILLY GROVE SUD
LIPAN CREEK FCD
LIPAN-KICKAPOO WATER CONSERVATION DISTRICT
LITTLE CYPRESS UTILITY DISTRICT
LITTLE YORK MUD
LITTLE YORK MUD
LIVE OAK CREEK MUD 1 OF TARRANT COUNTY
LIVE OAK UWCD
LLANO COUNTY FWSD 1
LLANO COUNTY MUD 1
LLANO ESTACADO UWCD
LLANO ESTACADO WATER DISTRIC

In [12]:
empty_reference_df

,SYSTEM_ID,SYSTEM_NAME,Status,URL
0,1011901,3 B&J MUD,INACTIVE,http://www14.tceq.texas.gov/iwud/dist/index.cf...
1,995951,"ABLES SPRINGS SUD OF HUNT, KAUFMAN, AND VAN ZA...",INACTIVE,http://www14.tceq.texas.gov/iwud/dist/index.cf...
2,980000,ACRES HOMES ID,DELETED/DISSOLVED,http://www14.tceq.texas.gov/iwud/dist/index.cf...
3,980500,ACTON MUD,DELETED/DISSOLVED,http://www14.tceq.texas.gov/iwud/dist/index.cf...
4,980502,ACTON MUD,ACTIVE,http://www14.tceq.texas.gov/iwud/dist/index.cf...
5,982000,ADAMS GARDENS IRRIGATION DISTRICT 19,ACTIVE,http://www14.tceq.texas.gov/iwud/dist/index.cf...
6,985000,ADDICKS UTILITY DISTRICT,ACTIVE,http://www14.tceq.texas.gov/iwud/dist/index.cf...
7,5328550,AGUA SUD,ACTIVE,http://www14.tceq.texas.gov/iwud/dist/index.cf...
8,985250,AIRLINE IMPROVEMENT DISTRICT,ACTIVE,http://www14.tceq.texas.gov/iwud/dist/index.cf...
9,985500,ALAMO CONSERVATION & REUSE DISTRICT,DELETED/DISSOLVED,http://www14.tceq.texas.gov/iwud/dist/index.cf...


In [11]:
        print(doc_id_links[audit].text)
        doc_id_links[audit].click()           
        avars = []; avals = []
        for var in driver.find_elements_by_css_selector(audit_vars_css):
            avars.append(var.text)
        for var in driver.find_elements_by_css_selector(audit_vals_css):
            avals.append(var.text)
        temp_audit = pd.DataFrame(avals).T
        temp_audit.columns = avars
        temp_audit['DOC_ID'] = driver.find_element_by_css_selector(get_doc_id).text
        temp_audit['DOC_ID_LIST_PAGE'] = doc_id_list[audit]
        temp_audit['DOC_LINK'] = doc_url_list[audit]
        temp_audit['SYSTEM_NAME'] = district_name_list[audit]
                        #temp_audit,index=avars,columns = [doc_id])
                        #temp_audit = temp_audit.append(pd.DataFrame([id],index=['System ID'],columns=[doc_id]))
                        #temp_audit = temp_audit.append(pd.DataFrame([doc_link],index=['Doc_Link'],columns=[doc_id]))
        empty_audit_info = pd.concat([empty_audit_info,temp_audit],axis=0, join='outer', ignore_index=True)
                    #empty_audit_info.to_csv('../Input/tceq_audits/fort_bend_audits.csv')
        time.sleep(1)
        driver.back()
        doc_id_links = driver.find_elements_by_css_selector('.iwud+ .iwud a')

In [14]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
#complete_doc_set = pd.DataFrame()
complete_doc_set = pd.read_csv('../../input/tceq_audits/master_doc_reference.csv',index_col=0)
empty_district_info = pd.DataFrame()

driver = webdriver.Chrome()  # Optional argument, if not specified will search path.
base_page = 'http://www14.tceq.texas.gov/iwud/document/index.cfm?fuseaction=listdocumentsbytype&Command=LIST'
driver.get(base_page)
#CountyCode

audit_vars_css = 'td.iwud:nth-child(1)'
audit_vals_css = '.iwud+ td'
get_doc_id = 'table:nth-child(6) tr:nth-child(1) .iwuddata'
access_more_docs = '.iwudgreyrow~ tr a img'
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
#
# select by value 
#select.select_by_value('1')
#wait = WebDriverWait(driver, 10)
select_doc_type = Select(driver.find_element_by_id('DocType'))
select_doc_type.select_by_visible_text('AUDIT REPORT')
driver.find_element_by_css_selector('.iwudstatus+ td input').click()
should_restart = True
while should_restart:
    should_restart = False
    doc_id_links = driver.find_elements_by_css_selector('.iwud+ .iwud a')
    district_links = driver.find_elements_by_css_selector('.iwud:nth-child(2) a')
    site_id = driver.find_elements_by_css_selector('form+ table tr :nth-child(3)')
    site_ids = []
    for site in site_id[1:]:
        site_ids.append(site.text)
    district_names = []
    doc_ids = []
    doc_urls = []
    not_blank = []
    for s in site_ids:
        not_blank.append(s != ' ')
    doc_id_links = [i for indx,i in enumerate(doc_id_links) if not_blank[indx]]
    for d in range(len(district_links)):
        district_names.append(district_links[d].text)
    for d in range(len(doc_id_links)):
        doc_ids.append(doc_id_links[d].text)
        doc_urls.append(doc_id_links[d].get_attribute('href'))   
    temp_doc_set = pd.DataFrame({'District Name': district_names,
     'DOC_ID': doc_ids,'DOC_URL':doc_urls})
    complete_doc_set = complete_doc_set.append(temp_doc_set)
    complete_doc_set.to_csv('../../input/tceq_audits/master_financial_doc_reference.csv')
    if len(driver.find_elements_by_css_selector("img[alt='Next page']"))!=0:
        driver.find_element_by_name('next').click()
        should_restart = True